### Construcción de un Sistema RAG con LangChain y FAISS
Introducción a RAG (Generación Aumentada por Recuperación)
RAG combina el poder de los sistemas de recuperación con modelos de IA generativa. En lugar de depender únicamente de los datos de entrenamiento del modelo, RAG:

1. Recupera documentos relevantes de una base de conocimiento
2. Usa estos documentos como contexto para el LLM
3. Genera respuestas basadas tanto en el contexto recuperado como en el conocimiento del modelo

### FAISS 
https://github.com/facebookresearch/faiss

FAISS es una biblioteca para búsqueda eficiente de similitud y agrupamiento de vectores densos.

Ventajas clave:
1. Búsqueda de similitud extremadamente rápida
2. Eficiente en memoria
3. Soporta aceleración GPU
4. Puede manejar millones de vectores

Cómo funciona:
- Indexa vectores para búsqueda rápida de vecinos más cercanos
- Devuelve los vectores más similares basados en métricas de distancia

In [1]:
# Cargar librerías necesarias
import os  # Para variables de entorno
from dotenv import load_dotenv  # Para cargar variables desde archivo .env
import numpy as np  # Para operaciones matemáticas con vectores
import warnings  # Para manejo de advertencias
warnings.filterwarnings('ignore')  # Suprimir advertencias

# Importaciones principales de LangChain
from langchain_core.documents import Document  # Clase para representar documentos
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate  # Para plantillas de prompts
from langchain_core.runnables import (
    RunnablePassthrough,  # Para pasar datos sin modificar
 
)
from langchain_core.output_parsers import StrOutputParser  # Para convertir salida a string
from langchain_core.messages import HumanMessage, AIMessage  # Para mensajes de chat

# Importaciones específicas de LangChain
from langchain_text_splitters import RecursiveCharacterTextSplitter  # Para dividir texto
from langchain_openai import OpenAIEmbeddings, ChatOpenAI  # Para embeddings y chat de OpenAI
from langchain_community.vectorstores import FAISS  # Base de datos vectorial FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader  # Para cargar documentos
from langchain.chains import create_retrieval_chain  # Para crear cadenas de recuperación
from langchain.chains.combine_documents import create_stuff_documents_chain  # Para combinar documentos

# Cargar variables de entorno desde archivo .env
load_dotenv()

True

### Ingesta y Procesamiento de Datos

In [2]:
# Crear documentos de muestra con información sobre IA y tecnologías relacionadas
sample_documents = [
    Document(
        page_content="""
        La Inteligencia Artificial (IA) es la simulación de la inteligencia humana en máquinas.
        Estos sistemas están diseñados para pensar como humanos e imitar sus acciones.
        La IA se puede categorizar en IA estrecha e IA general.
        """,
        metadata={"source": "Introducción IA", "page": 1, "topic": "IA"}  # Metadatos con información del documento
    ),
    Document(
        page_content="""
        El Aprendizaje Automático es un subconjunto de la IA que permite a los sistemas aprender de los datos.
        En lugar de ser programados explícitamente, los algoritmos de ML encuentran patrones en los datos.
        Los tipos comunes incluyen aprendizaje supervisado, no supervisado y por refuerzo.
        """,
        metadata={"source": "Fundamentos ML", "page": 1, "topic": "ML"}
    ),
    Document(
        page_content="""
        El Aprendizaje Profundo es un subconjunto del aprendizaje automático basado en redes neuronales artificiales.
        Utiliza múltiples capas para extraer progresivamente características de alto nivel de la entrada cruda.
        El aprendizaje profundo ha revolucionado la visión por computadora, PLN y reconocimiento de voz.
        """,
        metadata={"source": "Aprendizaje Profundo", "page": 1, "topic": "DL"}
    ),
    Document(
        page_content="""
        El Procesamiento de Lenguaje Natural (PLN) es una rama de la IA que ayuda a las computadoras a entender el lenguaje humano.
        Combina la lingüística computacional con modelos de aprendizaje automático y aprendizaje profundo.
        Las aplicaciones incluyen chatbots, traducción, análisis de sentimientos y resumen de texto.
        """,
        metadata={"source": "Resumen PLN", "page": 1, "topic": "PLN"}
    )
]

print(sample_documents)

[Document(metadata={'source': 'Introducción IA', 'page': 1, 'topic': 'IA'}, page_content='\n        La Inteligencia Artificial (IA) es la simulación de la inteligencia humana en máquinas.\n        Estos sistemas están diseñados para pensar como humanos e imitar sus acciones.\n        La IA se puede categorizar en IA estrecha e IA general.\n        '), Document(metadata={'source': 'Fundamentos ML', 'page': 1, 'topic': 'ML'}, page_content='\n        El Aprendizaje Automático es un subconjunto de la IA que permite a los sistemas aprender de los datos.\n        En lugar de ser programados explícitamente, los algoritmos de ML encuentran patrones en los datos.\n        Los tipos comunes incluyen aprendizaje supervisado, no supervisado y por refuerzo.\n        '), Document(metadata={'source': 'Aprendizaje Profundo', 'page': 1, 'topic': 'DL'}, page_content='\n        El Aprendizaje Profundo es un subconjunto del aprendizaje automático basado en redes neuronales artificiales.\n        Utiliza m

In [3]:
# División de texto en chunks más pequeños
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,  # Tamaño máximo de cada chunk en caracteres
    chunk_overlap=50,  # Superposición entre chunks para mantener contexto
    length_function=len,  # Función para medir la longitud del texto
    separators=[" "]  # Separadores para dividir (espacios en este caso)
)

# Dividir los documentos en chunks más pequeños para mejor procesamiento
chunks = text_splitter.split_documents(sample_documents)
print(chunks[0])  # Mostrar el primer chunk
print(chunks[1])  # Mostrar el segundo chunk

page_content='La Inteligencia Artificial (IA) es la simulación de la inteligencia humana en máquinas.
        Estos sistemas están diseñados para pensar como humanos e imitar sus acciones.
        La IA se puede categorizar en IA estrecha e IA general.' metadata={'source': 'Introducción IA', 'page': 1, 'topic': 'IA'}
page_content='El Aprendizaje Automático es un subconjunto de la IA que permite a los sistemas aprender de los datos.
        En lugar de ser programados explícitamente, los algoritmos de ML encuentran patrones en los datos.
        Los tipos comunes incluyen aprendizaje supervisado, no supervisado y por refuerzo.' metadata={'source': 'Fundamentos ML', 'page': 1, 'topic': 'ML'}


In [4]:
print(f"Creados {len(chunks)} chunks a partir de {len(sample_documents)} documentos")
print("\nEjemplo de chunk:")
print(f"Contenido: {chunks[0].page_content}")
print(f"Metadatos: {chunks[0].metadata}")

Creados 4 chunks a partir de 4 documentos

Ejemplo de chunk:
Contenido: La Inteligencia Artificial (IA) es la simulación de la inteligencia humana en máquinas.
        Estos sistemas están diseñados para pensar como humanos e imitar sus acciones.
        La IA se puede categorizar en IA estrecha e IA general.
Metadatos: {'source': 'Introducción IA', 'page': 1, 'topic': 'IA'}


In [5]:
### Cargar los modelos de embedding
import os  # Para manejo de variables de entorno
load_dotenv()  # Cargar variables desde .env

# Configurar la API key de OpenAI desde las variables de entorno
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [6]:
# Inicializar embeddings de OpenAI con el modelo más reciente

embeddings=OpenAIEmbeddings(
    model="text-embedding-3-small",  # Modelo de embedding pequeño y eficiente
    dimensions=1536  # Dimensiones del vector de embedding (1536 para este modelo)
)

# Ejemplo: crear un embedding para un texto individual
sample_text="¿Qué es el aprendizaje automático?"
sample_embedding=embeddings.embed_query(sample_text)  # Convertir texto a vector numérico
sample_embedding  # Mostrar el vector resultante (lista de números)

[-0.02507798746228218,
 -0.018135398626327515,
 -0.03354456275701523,
 -0.010904945433139801,
 0.01364811509847641,
 -0.006548894103616476,
 -0.010007488541305065,
 0.01752580516040325,
 -0.036711059510707855,
 0.05462632700800896,
 -0.011836268939077854,
 0.008026311174035072,
 -0.008458105847239494,
 -0.029920868575572968,
 0.013749714009463787,
 0.008280307985842228,
 -0.0036236930172890425,
 -0.0044195507653057575,
 0.021877624094486237,
 -0.040233153849840164,
 -0.01754273846745491,
 -0.007006088737398386,
 0.07003548741340637,
 -0.02844768390059471,
 0.0064853946678340435,
 -0.09807677567005157,
 -0.004825946409255266,
 0.01325865276157856,
 -0.02534891851246357,
 -0.03498387709259987,
 -0.020438306033611298,
 -0.023994266986846924,
 -0.022199353203177452,
 0.01976098120212555,
 0.05235728621482849,
 0.010803346522152424,
 -0.023621737957000732,
 0.01571395806968212,
 -0.03144485130906105,
 0.009203164838254452,
 0.006840990856289864,
 -0.0031072322744876146,
 0.01043081749230623

In [7]:
# Crear embeddings para múltiples textos de una vez (procesamiento en lote)
texts=["IA","Aprendizaje automático","Aprendizaje Profundo","Red Neuronal"]
batch_embeddings=embeddings.embed_documents(texts)  # Procesar todos los textos juntos
print(batch_embeddings[0])  # Mostrar el primer embedding del lote

[-0.018640760332345963, -0.02737336792051792, -0.009053210727870464, 0.028701579198241234, -0.04091501608490944, -0.005446430295705795, 0.01279357634484768, -0.015488166362047195, -0.02027430757880211, -0.001776673598214984, 0.0016803019680082798, -0.014908027835190296, -0.026121491566300392, -0.02737336792051792, 0.01818275637924671, -0.0005925425793975592, -0.023388734087347984, -0.023236066102981567, 0.04491491988301277, 0.03624337911605835, 0.025984089821577072, 0.05166284367442131, -0.028640512377023697, 0.01237373985350132, 0.014877494424581528, -0.016747677698731422, -0.02696116454899311, 0.029342785477638245, 0.01865602657198906, -0.014282089658081532, 0.06296027451753616, -0.025144414976239204, 0.02198418788611889, -0.02335819974541664, 0.005331929307430983, 0.030457261949777603, -0.024625344201922417, -0.006919675972312689, -0.009053210727870464, 0.013610349968075752, 0.03868606686592102, -0.0019045330118387938, 0.011625666171312332, 0.041159287095069885, -0.00300565082579851

In [8]:
print(batch_embeddings[1])

[-0.017638757824897766, -0.02764216996729374, -0.02356998436152935, 0.002711101435124874, -0.01518216822296381, 0.011475152336061, 0.013931741937994957, 0.047803912311792374, -0.03288732096552849, 0.026159362867474556, 0.013433785177767277, 0.001196481054648757, -0.021002737805247307, -0.03968167304992676, 0.03496767580509186, 0.010971661657094955, 0.008764050900936127, -0.012858367525041103, 0.023968350142240524, 0.011038056574761868, 0.0023016699124127626, -0.013677230104804039, 0.05404498055577278, -0.020626503974199295, 0.026070836931467056, -0.07422885298728943, 0.006224467884749174, 0.03768984228372574, -0.029589736834168434, -0.01884492114186287, 0.010523499920964241, -0.025362631306052208, -0.024986397475004196, 0.031161069869995117, 0.02675691246986389, 0.004503748379647732, -0.019907230511307716, 0.02421179600059986, -0.05369087681174278, 0.008144370280206203, 0.03890707343816757, -0.002365297870710492, 0.027265936136245728, 0.03799968585371971, -0.015082577243447304, -0.0133

In [9]:
### Comparar Embeddings usando similitud coseno

def compare_embeddings(text1:str,text2:str):
    """Comparar la similitud semántica de 2 textos usando embeddings"""

    # Generar embeddings para ambos textos y convertir a arrays numpy
    emb1=np.array(embeddings.embed_query(text1))
    emb2=np.array(embeddings.embed_query(text2))

    # Calcular la puntuación de similitud usando similitud coseno
    # Fórmula: (A·B) / (||A|| * ||B||) donde · es producto punto y |||| es norma
    similarity=np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similarity  # Devuelve valor entre -1 y 1 (1 = idénticos, 0 = no relacionados)

In [10]:
# Probar similitud semántica entre diferentes textos
print("\nEjemplos de Similitud Semántica:")
print(f"'IA' vs 'Inteligencia Artificial': {compare_embeddings('IA', 'Inteligencia Artificial'):.3f}")


Ejemplos de Similitud Semántica:
'IA' vs 'Inteligencia Artificial': 0.305


In [11]:
print(f"'IA' vs 'Pizza': {compare_embeddings('IA', 'Pizza'):.3f}")  # Conceptos no relacionados

'IA' vs 'Pizza': 0.269


In [12]:
print(f"'Aprendizaje Automático' vs 'ML': {compare_embeddings('Aprendizaje Automático', 'ML'):.3f}")  # Conceptos relacionados

'Aprendizaje Automático' vs 'ML': 0.249


### Crear Almacén Vectorial FAISS

In [13]:
# Crear almacén vectorial FAISS a partir de los chunks de documentos
vectorstore=FAISS.from_documents(
    documents=chunks,  # Los chunks de texto a almacenar
    embedding=embeddings  # Modelo de embeddings a usar para convertir texto a vectores
)
print(f"Almacén vectorial creado con {vectorstore.index.ntotal} vectores")

Almacén vectorial creado con 4 vectores


In [14]:
vectorstore

In [15]:
# Guardar el almacén vectorial para uso posterior
vectorstore.save_local("faiss_index")  # Guarda el índice en directorio local
print("Almacén vectorial guardado en el directorio 'faiss_index'")

Almacén vectorial guardado en el directorio 'faiss_index'


In [16]:
# Cargar almacén vectorial previamente guardado
loaded_vectorstore=FAISS.load_local(
    "faiss_index",  # Directorio donde está guardado el índice
    embeddings,  # Modelo de embeddings (debe ser el mismo usado para crear)
    allow_dangerous_deserialization=True  # Permitir deserialización (usar con cuidado)
)

print(f"Almacén vectorial cargado contiene {loaded_vectorstore.index.ntotal} vectores")

Almacén vectorial cargado contiene 4 vectores


In [17]:
# Búsqueda por similitud - encontrar documentos más relevantes
query="¿Qué es el aprendizaje profundo?"

# Buscar los 3 documentos más similares a la consulta
results=vectorstore.similarity_search(query,k=3)  # k=3 significa devolver top 3 resultados
print(results)

[Document(id='e013b682-41f1-4d8b-862b-c743e43126ed', metadata={'source': 'Aprendizaje Profundo', 'page': 1, 'topic': 'DL'}, page_content='El Aprendizaje Profundo es un subconjunto del aprendizaje automático basado en redes neuronales artificiales.\n        Utiliza múltiples capas para extraer progresivamente características de alto nivel de la entrada cruda.\n        El aprendizaje profundo ha revolucionado la visión por computadora, PLN y reconocimiento de voz.'), Document(id='cc99b92e-e6a0-4c32-8f73-6dbbf913e8e3', metadata={'source': 'Fundamentos ML', 'page': 1, 'topic': 'ML'}, page_content='El Aprendizaje Automático es un subconjunto de la IA que permite a los sistemas aprender de los datos.\n        En lugar de ser programados explícitamente, los algoritmos de ML encuentran patrones en los datos.\n        Los tipos comunes incluyen aprendizaje supervisado, no supervisado y por refuerzo.'), Document(id='2a94efe7-a8c4-4b96-ba8a-a96dcfd561f6', metadata={'source': 'Resumen PLN', 'page'

In [18]:
print(f"Consulta: {query}\n")
print("Top 3 chunks más similares:")
for i, doc in enumerate(results):
    print(f"\n{i+1}. Fuente: {doc.metadata['source']}")
    print(f"   Contenido: {doc.page_content[:200]}...")  # Mostrar solo primeros 200 caracteres

Consulta: ¿Qué es el aprendizaje profundo?

Top 3 chunks más similares:

1. Fuente: Aprendizaje Profundo
   Contenido: El Aprendizaje Profundo es un subconjunto del aprendizaje automático basado en redes neuronales artificiales.
        Utiliza múltiples capas para extraer progresivamente características de alto nivel...

2. Fuente: Fundamentos ML
   Contenido: El Aprendizaje Automático es un subconjunto de la IA que permite a los sistemas aprender de los datos.
        En lugar de ser programados explícitamente, los algoritmos de ML encuentran patrones en l...

3. Fuente: Resumen PLN
   Contenido: El Procesamiento de Lenguaje Natural (PLN) es una rama de la IA que ayuda a las computadoras a entender el lenguaje humano.
        Combina la lingüística computacional con modelos de aprendizaje auto...


In [19]:
### Búsqueda por similitud con puntuaciones
# Obtener documentos similares junto con sus puntuaciones de similitud
results_with_scores=vectorstore.similarity_search_with_score(query,k=3)

print("\n\nBúsqueda por similitud con puntuaciones:")
for doc, score in results_with_scores:
    print(f"\nPuntuación: {score:.3f}")  # Puntuación más baja = más similar en FAISS
    print(f"Fuente: {doc.metadata['source']}")
    print(f"Vista previa del contenido: {doc.page_content[:100]}...")



Búsqueda por similitud con puntuaciones:

Puntuación: 0.523
Fuente: Aprendizaje Profundo
Vista previa del contenido: El Aprendizaje Profundo es un subconjunto del aprendizaje automático basado en redes neuronales arti...

Puntuación: 1.097
Fuente: Fundamentos ML
Vista previa del contenido: El Aprendizaje Automático es un subconjunto de la IA que permite a los sistemas aprender de los dato...

Puntuación: 1.175
Fuente: Resumen PLN
Vista previa del contenido: El Procesamiento de Lenguaje Natural (PLN) es una rama de la IA que ayuda a las computadoras a enten...


In [20]:
chunks

[Document(metadata={'source': 'Introducción IA', 'page': 1, 'topic': 'IA'}, page_content='La Inteligencia Artificial (IA) es la simulación de la inteligencia humana en máquinas.\n        Estos sistemas están diseñados para pensar como humanos e imitar sus acciones.\n        La IA se puede categorizar en IA estrecha e IA general.'),
 Document(metadata={'source': 'Fundamentos ML', 'page': 1, 'topic': 'ML'}, page_content='El Aprendizaje Automático es un subconjunto de la IA que permite a los sistemas aprender de los datos.\n        En lugar de ser programados explícitamente, los algoritmos de ML encuentran patrones en los datos.\n        Los tipos comunes incluyen aprendizaje supervisado, no supervisado y por refuerzo.'),
 Document(metadata={'source': 'Aprendizaje Profundo', 'page': 1, 'topic': 'DL'}, page_content='El Aprendizaje Profundo es un subconjunto del aprendizaje automático basado en redes neuronales artificiales.\n        Utiliza múltiples capas para extraer progresivamente cara

In [21]:
### Búsqueda con filtrado por metadatos
# Filtrar resultados basándose en metadatos específicos
filter_dict={"topic":"ML"}  # Solo buscar en documentos con topic="ML"
filtered_results=vectorstore.similarity_search(
    query,  # La misma consulta
    k=3,  # Máximo 3 resultados
    filter=filter_dict  # Aplicar filtro de metadatos
)
print(filtered_results)

[Document(id='cc99b92e-e6a0-4c32-8f73-6dbbf913e8e3', metadata={'source': 'Fundamentos ML', 'page': 1, 'topic': 'ML'}, page_content='El Aprendizaje Automático es un subconjunto de la IA que permite a los sistemas aprender de los datos.\n        En lugar de ser programados explícitamente, los algoritmos de ML encuentran patrones en los datos.\n        Los tipos comunes incluyen aprendizaje supervisado, no supervisado y por refuerzo.')]


In [22]:
len(filtered_results)

1

### Construir Cadena RAG con LCEL (Lenguaje de Expresión de LangChain)

In [28]:
# Configurar LLM usando GROQ (alternativa rápida y económica a OpenAI)
from langchain.chat_models import init_chat_model

# Configurar API key de GROQ desde variables de entorno
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

# Inicializar modelo de chat usando GROQ con Gemma2
llm=init_chat_model(model="groq:llama-3.1-8b-instant")  # Gemma2 9B es rápido y eficiente
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024C8EA66090>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024C8EA67680>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [29]:
llm.invoke("hola")

AIMessage(content='Hola. ¿En qué puedo ayudarte?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 37, 'total_tokens': 47, 'completion_time': 0.015204205, 'prompt_time': 0.002253906, 'queue_time': 0.202341134, 'total_time': 0.017458111}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d4c6f180-6b86-4124-a571-816b552f3d91-0', usage_metadata={'input_tokens': 37, 'output_tokens': 10, 'total_tokens': 47})

In [30]:
# 1. Cadena RAG Simple usando LCEL
# Crear plantilla de prompt que instruye al modelo sobre cómo usar el contexto
simple_prompt = ChatPromptTemplate.from_template("""Responde la pregunta basándote únicamente en el siguiente contexto:
Contexto: {context}

Pregunta: {question}

Respuesta:""")

In [31]:
# Configurar recuperador básico a partir del almacén vectorial
retriever=vectorstore.as_retriever(
    search_type="similarity",  # Tipo de búsqueda por similitud
    search_kwargs={"k":3}  # Recuperar los 3 documentos más relevantes
)

In [32]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024C8EB3B8F0>, search_kwargs={'k': 3})

In [33]:
from typing import List
# Formatear documentos para inserción en el prompt
def format_docs(docs: List[Document]) -> str:
    """Formatear documentos para inserción en el prompt"""
    formatted = []  # Lista para almacenar documentos formateados
    for i, doc in enumerate(docs):  # Iterar sobre cada documento
        source = doc.metadata.get('source', 'Desconocido')  # Obtener fuente del documento
        # Crear formato legible con numeración y fuente
        formatted.append(f"Documento {i+1} (Fuente: {source}):\n{doc.page_content}")
    return "\n\n".join(formatted)  # Unir todos los documentos con doble salto de línea

In [34]:
# Construir cadena RAG simple usando sintaxis de LCEL
simple_rag_chain=(
    {"context":retriever | format_docs,"question":RunnablePassthrough() }  # Recuperar contexto y pasar pregunta
    | simple_prompt  # Aplicar plantilla de prompt
    | llm  # Enviar al modelo de lenguaje
    | StrOutputParser()  # Convertir respuesta a string
)

In [35]:
simple_rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024C8EB3B8F0>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Responde la pregunta basándote únicamente en el siguiente contexto:\nContexto: {context}\n\nPregunta: {question}\n\nRespuesta:'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024C8EA66090>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024C8EA67680>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [36]:
### Cadena RAG Conversacional

# Crear prompt conversacional que mantiene historial de chat
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente de IA útil. Usa el contexto proporcionado para responder preguntas."),
    ("placeholder", "{chat_history}"),  # Marcador para historial de conversación
    ("human", "Contexto: {context}\n\nPregunta: {input}"),  # Mensaje del usuario con contexto
])

In [37]:
def create_conversational_rag():
    """Crear una cadena RAG conversacional con memoria"""
    return (
        RunnablePassthrough.assign(  # Asignar contexto dinámicamente
            context=lambda x: format_docs(retriever.invoke(x["input"]))  # Recuperar y formatear documentos
        )
        | conversational_prompt  # Aplicar prompt conversacional
        | llm  # Enviar al modelo
        | StrOutputParser()  # Parsear respuesta
    )

conversational_rag = create_conversational_rag()  # Crear instancia de la cadena

In [38]:
conversational_rag

RunnableAssign(mapper={
  context: RunnableLambda(lambda x: format_docs(retriever.invoke(x['input'])))
})
| ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag=

In [39]:
### Cadena RAG con streaming (transmisión en tiempo real)
# Esta cadena permite ver la respuesta generándose token por token
streaming_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}  # Preparar datos
    | simple_prompt  # Aplicar prompt
    | llm  # Enviar al modelo (sin parser para streaming)
)

print("¡Cadenas RAG modernas creadas exitosamente!")
print("Cadenas disponibles:")
print("- simple_rag_chain: Preguntas y respuestas básicas")
print("- conversational_rag: Mantiene historial de conversación")
print("- streaming_rag_chain: Soporta transmisión de tokens")

¡Cadenas RAG modernas creadas exitosamente!
Cadenas disponibles:
- simple_rag_chain: Preguntas y respuestas básicas
- conversational_rag: Mantiene historial de conversación
- streaming_rag_chain: Soporta transmisión de tokens


In [40]:
# Función de prueba para diferentes tipos de cadenas
def test_rag_chains(question: str):
    """Probar todas las variantes de cadenas RAG"""
    print(f"Pregunta: {question}")
    print("=" * 80)
    
    # 1. RAG Simple
    print("\n1. Cadena RAG Simple:")
    answer = simple_rag_chain.invoke(question)  # Obtener respuesta completa
    print(f"Respuesta: {answer}")

    # 2. RAG con Streaming
    print("\n2. RAG con Streaming:")
    print("Respuesta: ", end="", flush=True)
    # Transmitir respuesta token por token
    for chunk in streaming_rag_chain.stream(question):
        print(chunk.content, end="", flush=True)  # Imprimir cada token sin salto de línea
    print()  # Salto de línea final

In [41]:
# Probar las cadenas con una pregunta específica
test_rag_chains("¿Cuál es la diferencia entre IA y aprendizaje automático?")

Pregunta: ¿Cuál es la diferencia entre IA y aprendizaje automático?

1. Cadena RAG Simple:
Respuesta: La diferencia entre IA y aprendizaje automático (ML) se puede establecer basándose en los documentos proporcionados:

- La Inteligencia Artificial (IA) es la simulación de la inteligencia humana en máquinas (Documento 2). Esto implica la capacidad de los sistemas para pensar y actuar como humanos.
- El Aprendizaje Automático (ML) es un subconjunto de la IA que permite a los sistemas aprender de los datos y encontrar patrones en ellos (Documento 1). Esto implica la capacidad de los sistemas para aprender y mejorar su desempeño sin ser programados explícitamente.

En resumen, toda la IA no es necesariamente aprendizaje automático, pero todo el aprendizaje automático es un subconjunto de la IA. El aprendizaje automático es un enfoque específico dentro de la IA que se centra en el aprendizaje de patrones en datos, mientras que la IA es un concepto más amplio que incluye la simulación de la

In [42]:
# Probar con múltiples preguntas
test_questions = [
    "¿Cuál es la diferencia entre IA y Aprendizaje Automático?",
    "Explica el aprendizaje profundo en términos simples",
    "¿Cómo funciona el PLN?"
]

# Iterar sobre cada pregunta y probar las cadenas
for question in test_questions:
    print("\n" + "=" * 80 + "\n")
    test_rag_chains(question)



Pregunta: ¿Cuál es la diferencia entre IA y Aprendizaje Automático?

1. Cadena RAG Simple:
Respuesta: Según los documentos proporcionados, la diferencia entre IA y Aprendizaje Automático es que la IA es un concepto más amplio que abarca la simulación de la inteligencia humana en máquinas, mientras que el Aprendizaje Automático es un subconjunto de la IA que permite a los sistemas aprender de los datos.

En otras palabras, la IA es la capacidad en general de los sistemas de pensar y actuar como humanos, mientras que el Aprendizaje Automático es una técnica específica dentro de la IA que se enfoca en el aprendizaje de patrones en los datos sin necesidad de programación explícita.

En resumen:

- IA: simulación de la inteligencia humana en máquinas (IA estrecha y general)
- Aprendizaje Automático: subconjunto de la IA que permite a los sistemas aprender de los datos (aprendizaje supervisado, no supervisado y por refuerzo)

2. RAG con Streaming:
Respuesta: Según los documentos proporcion

In [43]:
## Ejemplo conversacional
print("\n3. Ejemplo de RAG Conversacional:")
chat_history = []  # Inicializar historial vacío

# Primera pregunta
q1 = "¿Qué es el aprendizaje automático?"
a1 = conversational_rag.invoke({  # Invocar cadena conversacional
    "input": q1,  # Pregunta actual
    "chat_history": chat_history  # Historial de conversación
})

print(f"P1: {q1}")
print(f"R1: {a1}")


3. Ejemplo de RAG Conversacional:
P1: ¿Qué es el aprendizaje automático?
R1: Según el contexto proporcionado, el aprendizaje automático es un subconjunto de la Inteligencia Artificial (IA) que permite a los sistemas aprender de los datos. En lugar de ser programados explícitamente, los algoritmos de aprendizaje automático encuentran patrones en los datos y se entrenan para realizar tareas específicas. Esto se menciona en el Documento 1 (Fuente: Fundamentos ML).


In [44]:
# Actualizar historial con la pregunta y respuesta anteriores
chat_history.extend([
    HumanMessage(content=q1),  # Mensaje del usuario
    AIMessage(content=a1)  # Respuesta del asistente
])

In [ ]:
# Pregunta de seguimiento (que depende del contexto anterior)
q2 = "¿En qué se diferencia de la programación tradicional?"
a2 = conversational_rag.invoke({  # El modelo puede usar el historial para entender "se diferencia"
    "input": q2,  # Nueva pregunta
    "chat_history": chat_history  # Historial incluye contexto de ML
})
print(f"\nP2: {q2}")
print(f"R2: {a2}")